# **Challenge 3 - Neural Networks**
* Sahil Gill, Luke Ingram
* MATH 318 - Winter 2023

In [ ]:
import cv2 
import tensorflow as tf
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

## **Section 1 - Why Neural Nets?**

From the previous challenge, we can conclude that classifying these images is difficult. Despite our best attempts at expanding the K-Nearest-Neighbors algorithm, our model's accuracy was is not what's desired from a classifier. So, once again, we try a more sophisticated approach. 

Which brings us to Neural Networks. Neural networks are complex, but flexible, and have soared in popularity over the last 10-15 years. One specific type of Neural Net, the **Convolutionary Neural Network** designed for image processing, is what we will be using to tackle the challenge of trash classification. 


## **Section 2 - The Architecture**

### **Section 2.1 - MobileNetV2**

TODO: INTRODUCE MOBILENETV2

### **Section 2.2 - Transfer Learning**

## **Section 3 - MobileNetV2 in Action**

Now it's time to put this into action. 

Before we begin, we need to load & normalize our data.

In [ ]:
#Load data 
dirName = "../src/data/archive/zero-indexed-files.txt"
imgPath = "../src/data/archive/Garbage_classification/load/"

df = pd.read_csv(dirName,sep=' ')

df['image'] = imgPath + df['image'].astype*(str)
df['image'] = df['image'].apply(lambda x: cv2.resize(cv2.imread(x),(224,224)))
print(df.head()) #DEBUG

train_X,test_X,train_Y,test_Y = train_test_split(df['image'],df['class'],
                                                 test_size=0.20,random_state=42,stratify=df['class'])


train_X,val_X,val_X,val_Y = train_test_split(train_X,train_Y,
                                               test_size=0.20,random_state=42,stratify=train_Y)

# Normalize data 
train_X = train_X/255.0 
test_X = test_X/255.0
val_X = val_X/255.0

### **Section 3.1 - Initializing MobileNetV2**

Keras provides us with an implementation of the MobileNetV2 network, we only have to specify the hyperparameters as follows: 

In [ ]:
#Fetch network from keras, & define custom params. 
# This is a pre-trained model,but we remove the last layer 
# & train it ourselves to fit out problem
model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape = None, # our images are already the required shape (244x244)
    alpha = 1.0, # using default input width
    include_top = True, #include fully-connected input layer
    weights = 'imagenet', # Default weights
    input_tensor = None, # using default input tensor structure
    pooling = None, # not using this feature
    classes = 1000, # default for MobileNet
    classifier_activation = 'softmax' #specify activation function of output layer
)   

# Make sure existing layers (convolutionary & pooling) remain untouched
for layer in model.layers: 
    layer.trainable = False 

### **Section 3.2 - Customized Output Layers**

TODO: Explain the transfer learning step & why we are using this many layers

In [ ]:
final_layer = model.layers[-2].output

output_layer = tf.keras.layers.Dense(144,activation='relu')(final_layer)
output_layer = tf.keras.layers.Dense(72,activation='relu')(final_layer)
output_layer = tf.keras.layers.Dense(7,activation='softmax')(final_layer)

# add it back to model
model = tf.keras.Model(inputs = model.layers[0].input,outputs = output_layer)

# Compile final model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='SGD', # Stochastic- Gradient - Descent
    metrics = ['accuracy']
)

### **Section 3.3 - Training the Model**

TODO: explain stochastic gradient descent

In [ ]:
# Convert to float for tensor conversion 
train_X = np.asarray(list(train_X),dtype='float32')
test_X = np.asarray(list(test_X),dtype='float32')
val_X = np.asarray(list(val_X),dtype='float32')


training_history = model.fit(train_X,train_Y,epochs = 30,validation_data=(val_X,val_Y))

plt.clf()
plt.rc('axes', axisbelow=True)
plt.grid(linestyle='dotted')
plt.plot(training_history['accuracy'],label='accuracy')
plt.plot(training_history['val_accuracy'],label = 'val_accuracy')
_ = plt.xlabel('Epoch')
_ = plt.ylabel('Accuracy')
_ = plt.title('Training & Validation Accuracy During Training')

plt.show() # TODO MAKE PLOT FOR TRAINING & VALIDATION ACCURACY


model.evaluate(test_X,test_Y) 

## **Section 4 - Results & Comparison with Previous Methods**

TODO: load a few classified results & display